In [1]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys

sys.path.append('../')
from Agents.A2CAgent_batch import Agent
from Boards.Speed_leedo_2p_v6 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

env = FullBoard()
agent0 = Agent(n_actions=env.action_size(), input_dim=env.state_size(), alpha=1e-8, max_val=env.max_val())
bot0 = Bot()

wins = []

output_dir_a = '../trainers_vs_intbot/model_output_vs_intbot/A2C_v7_batch/0002/actor/weights_130000.hdf5'
output_dir_c = '../trainers_vs_intbot/model_output_vs_intbot/A2C_v7_batch/0002/critic/weights_130000.hdf5'

# output_dir_a = '../model_output/A2C_v5/0001/actor/weights_16000.hdf5'
# output_dir_c = '../model_output/A2C_v5/0001/critic/weights_16000.hdf5'

agent0.load_model(output_dir_a, output_dir_c)

for ep in tqdm(range(0, 10000), ascii=True, unit="e"):
    s, _, game_over, player_turn = env.reset()
    episode_reward = [0.0, 0.0]

    time = 0
    while time < 360 or player_turn == 0:

        time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
        player_turn_temp = env.get_player_turn()
        env.roll_dice()[0]
        player_turn = env.get_player_turn()

        if player_turn == player_turn_temp:

            action_list = env.get_next_states(player_turn)

            if action_list:
                if player_turn == 1:
                    s_t = env.convert_state(player_turn)
                    action = agent0.act_test(s_t, action_list)
                else:
                    s_t = env.return_state()
                    action = bot0.act(state=s_t, p=player_turn)

                time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
                s_, reward, game_over, player_turn_temp = env.make_step(action)

                episode_reward[player_turn] += reward[player_turn]
       
    if episode_reward[1] > episode_reward[0]:
        wins.append(1)
    else:
        wins.append(0)

    if ep % 1000 == 0 and ep != 0:
        print(np.sum(wins))

2022-10-25 16:47:46.874424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 16:47:46.885928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 16:47:46.886704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 16:47:46.893752: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

176


 20%|##        | 2001/10000 [08:34<33:51,  3.94e/s]

345


 30%|###       | 3001/10000 [12:51<31:04,  3.75e/s]

504


 40%|####      | 4001/10000 [17:09<26:37,  3.75e/s]

683


 46%|####6     | 4633/10000 [19:52<21:49,  4.10e/s]

In [ ]:
np.sum(wins)

2322